# Working with NetCDF files 

One of the most common file formats within environmental science is NetCDF ([Network Common Data Form](https://www.unidata.ucar.edu/software/netcdf/)). 

This format allows for storage of multiple variables, over multiple dimensions (i.e. N-dimensional arrays).   
Files also contain the associated history and variable attributes. 

If you're not familiar with NetCDF, and would like to know more, there is a bit more general information at the bottom of this notebook.  
For now, we'll simply focus on how to access and work with these files in python ... 

#### Example dataset
http://xarray.pydata.org/en/stable/data-structures.html

<br>
<img src="../figures/dataset-diagram.png">


# NetCDF in python

There are a few different packages that can be used to access data from NetCDF files.  
These include: 

* [netCDF4](https://unidata.github.io/netcdf4-python/netCDF4/index.html)
  * Core netCDF4 package within python.  
* [iris](https://scitools.org.uk/iris/docs/latest/index.html) 
  * Developed for earth system data. 
  * Basic plots can be created within iris, with links to cartopy. 
* [xarray](http://xarray.pydata.org/en/stable/)
  * A higher-level package, with a pandas-like interface for netCDF. 
  * What we'll focus on here today.


## netCDF4

Contains everything you need to read/modify/create netCDF files. e.g. 

```python
from netCDF4 import Dataset

openfile = Dataset('../data/cefas_GETM_nwes.nc4')
bathymetry = openfile.variables['bathymetry'][:]
```

Variables are read into NumPy arrays (masked arrays if missing values specified). 

## xarray


* Alternative to plain netCDF4 access from python. 

* Brings the power of pandas to environmental sciences, by providing N-dimensional variants of the core pandas data structures.
  * Worth using for N-dimensional data, even when not reading netCDF files?

| Pandas | xarray  |
|---|---|
| 1-D Series  | DataArray  |
| DataFrame  | Dataset  |

DataArray uses names of dimensions making it easier to track than by using axis numbers. 

For example, if you want to average your DataArray (da) over time, it is possible to write `da.mean(dim='time')`  
You don't have to remember the index of the time axis.

Compare:
```python
# xarray style
>>> da.sel(time='2018-01-12').max(dim='ensemble')

# standard numpy style
>>> array[3, :, :].max(axis=2)
```

Without xarray, you need to first check which row refers to `time='2018-01-12'`, and which dimension is relevant for the ensemble. 

In the NumPy example, these choices are also not obvious to anyone reading the code at a later date. 

#### The main advantages of using xarray versus plain netCDF4 are:

* intelligent selection along labelled dimensions (and also indices)
* [groupby operations](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.groupby.html)
* data alignment across named axes
* IO (netcdf)
  * Attributes/metadata held with the dataset. 
* conversion from and to Pandas.DataFrames

## Xarray as Pandas for N dimensions

In [ ]:
# Import everything that we are going to need... but not more
import pandas as pd
import xarray as xr
import numpy as np

In [ ]:
pd_s = pd.Series(range(3), index=list('abc'))
print(pd_s)

In [ ]:
#convert 1D series to ND aware DataArray 
da = xr.DataArray(pd_s)
print(da)

In [ ]:
#convert DataFrame to ND aware DataArray
df = pd.DataFrame.from_dict({'A': [1, 2, 3], 'B': [4, 5, 6]},                         
                             orient='index', columns=['one', 'two', 'three'])
df

In [ ]:
ds = xr.Dataset.from_dataframe(df)
ds

---

# Lets open a netCDF file

Here's a dataset we prepared earlier (find in your data folder): `cefas_GETM_nwes.nc4`

In [ ]:
GETM = xr.open_dataset('../data/cefas_GETM_nwes.nc4')
GETM

We can extract information on the dimensions, coordinates and attributes of the dataset

In [ ]:
# List dimensions
GETM.dims

In [ ]:
# Extract coordinates
print(type(GETM.coords['latc']))
GETM.coords['latc'].shape

In [ ]:
# List name of dataset attributes
GETM.attrs.keys()

In [ ]:
# List variable names
GETM.data_vars.keys()

In [ ]:
# print variable attributes
print(GETM['h'].attrs)

## Extract variable from dataset

In [ ]:
temp = GETM['temp']
print(temp.shape)
temp

# Can also use: 
# GETM.temp

Check variable attributes, in the same way we access DataSet attributes

In [ ]:
print(temp.attrs)
print('Variable {} has units {}'.format(temp.attrs['long_name'], temp.attrs['units']))

### Accessing data values

Data can be subset using standard indexing methods. 

In [ ]:
temp[0, 0, 90, 100]

In [ ]:
temp.values[0, 0, 90, 100]

---

## Xarray Indexing and selecting data

Xarray offers a variety of ways to subset your data. 

From http://xarray.pydata.org/
<br>
<img src="../figures/xarray_indexing_table.png">

In [ ]:
#positional by integer
print( temp[0, 2, :, :].shape )
temp.dims

In [ ]:
# positional by label (coordinate value)
print( temp.loc['1996-02-02T01:00:00', 6, :, :].shape )

In [ ]:
# by name and integer
print( temp.isel(level=1, latc=90, lonc=100).shape )

In [ ]:
# by name and label
print( temp.sel(time='1996-02-02T01:00:00').shape )

### Define selection using nearest value

In examples above, you use the coordinate values to make the selection by label. 

If the value you want doesn't exist, it is possible to interpolate to the nearest index. e.g. 

In [ ]:
temp.sel(level=2, lonc=-5.0, latc=50.0, method='nearest')

In [ ]:
# Use tolerance for label selection 
# e.g. latc=-50 should not yield data
tol = 0.5

try:
    print(temp.sel(level=1, lonc=-5.0, latc=-50.0, method='nearest', tolerance=tol))
except KeyError:
    print(f'ERROR: outside tolerance of {tol}')

Note: Other `method` options available are: 
* Backfill / bfill - propagate values backward
* Pad / ffill - propagate values forward
* None - default, exact matches only

More information can be found in the xarray docs [here](http://xarray.pydata.org/en/stable/indexing.html). 

---
---

# Plotting

Xarray enables simple plotting, to easily view your data

In [ ]:
# But often, this will do
#GETM.temp.isel(time=0, level=0).plot();
GETM.temp.sel(time='1996-02-02T01:00:00', level=21).plot();

We can use other packages to create something neater e.g. for publication or presentation. 

Let's look an example with cartopy.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs

Define a general mapping function

In [ ]:
#def make_map(ds, var='', title=None, units=None):
def make_map():
    # create figure and axes instances
    fig = plt.figure()
    ax = fig.add_subplot(111, projection=ccrs.Stereographic(central_latitude=60))
    ax.set_extent([-10, 15, 49, 60], crs=ccrs.PlateCarree())
    
    gl = ax.gridlines(draw_labels=False)
    
    feature = cartopy.feature.NaturalEarthFeature(name='coastline',
                                                  category='physical',
                                                  scale='50m',
                                                  edgecolor='0.5',
                                                  facecolor='0.8')
    ax.add_feature(feature)
    return fig, ax

make_map();

Plot your chosen data on the map

In [ ]:
latc = GETM.coords['latc']
lonc = GETM.coords['lonc']

var = GETM['temp'].sel(time='1996-02-02T01:00:00', level=21)

# create arrays of coordinates for contourf
# lon2d, lat2d = np.meshgrid(lonc, latc)

fig, ax = make_map()
# draw filled contours.
h = ax.contourf(lonc, latc, var, 50, cmap=plt.cm.coolwarm, transform=ccrs.PlateCarree())

# add colorbar.
cbar = fig.colorbar(h)
# with unit label
cbar.set_label(var.units)

# add title
ax.set_title(f'A slice of {var.long_name}');

## Arithmetic operations

In [ ]:
top = GETM['temp'].isel(time=0, level=4)
bottom = GETM['temp'].isel(time=0, level=0)

diff = top - bottom

diff.plot()

### Calculate average along a dimension

In [ ]:
# average over time
time_ave = GETM['temp'].mean('time')

#average over time and level (vertical)
timelev_ave = GETM['temp'].mean(['time','level'])

timelev_ave.plot()

In [ ]:
#zonal average (vertical)
timelon_ave = GETM['temp'].mean(['time','lonc']).isel(level=4)

timelon_ave.plot()

## A dataset can easily be saved to a netCDF file

In [ ]:
ds = GETM[['temp']].mean('time','level')
ds.to_netcdf('../data/temp_avg_level_time.nc')

In [ ]:
print(type( GETM[['temp']]) )
print(type( GETM['temp'])   )

## Exercise

* Extract the bathymetry
* Extract the time averaged seabed temperature (level=0)
* Produce a scatter plot of depth vs. seabed temperature


In [ ]:
# bathy = GETM
GETM.data_vars.keys()
bathy=GETM['bathymetry']
bathy

temp_bott=GETM['temp'].mean('time').isel(level=0)
temp_bott

# bedtemp=GETM

# plt.scatter(  , ,marker='.')

In [ ]:
bathy = GETM['bathymetry']
bathy.plot();

In [ ]:
bedtemp=GETM['temp'].isel(level=0).mean('time')
bedtemp.plot()
bedtemp
#plt.scatter(bathy,bedtemp)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(bathy,bedtemp,marker='.',s=1.0)
ax.set_xlabel('bathymetry (m)')
ax.set_ylabel('Temp. diff (deg. C)');

---

## Import remote datasets

xarray supports OpenDAP. This means that a dataset can be accessed remotely and subsetted as needed. Only the selected parts are downloaded.

In [ ]:
# Naughty datasets might require decode_cf=False
# Here it just needed decode_times=False

naughty_data = xr.open_dataset(
      'http://iridl.ldeo.columbia.edu/SOURCES/.OSU/.PRISM/.monthly/dods',
      decode_times=False)
naughty_data

---
---

# More on the netCDF file format

* popular scientific file format for ocean and atmospere gridded datasets
* netCDF is a collection of formats for storing arrays

    * netCDF classic
        * more widespread
        * 2 GB file limit (if you don't use the unlimited dimension)
        * often preffered for distributing products

    * netCDF 64 bit offset
        * supports larger files

    * NetCDF4
        * based on HDF5
        * compression
        * multiple unlimited variables
        * new types inc. user defined
        * herarchical groups
        
        

* Developed by Unidata-UCAR with the aim of storing climate model data (3D+time)
* Auxilary information about each variable can be added
* Readable text equivalent called CDL (use ncdump/ncgen)
* Can be used with Climate and Forecast (CF) data convention
http://cfconventions.org/

## Data model:

* Dimensions:describe the axes of the data arrays.
* Variables: N-dimensional arrays of data.
* Attributes: annotate variables or files with small notes or supplementary metadata.

Example for an ocean model dataset:

* Dimensions
    * lat
    * lon
    * depth
    * time
* Variable
    * Temperature
    * Salinity
* Global Attibutes
    * Geographic grid type
    * History



* Variable attributes (Temperature)
    * Long_name: "sea water temperature" 
    * Missing_value: 1.09009E36
    * Units: deg. C
    * range: -2:50

## Tools for working with netCDF files


#### You can peek inside your netcdf file from the prompt window: 

```
$ ncdump data/cefas_GETM_nwes.nc4 | more
netcdf cefas_GETM_nwes {
dimensions:
        latc = 360 ;
        lonc = 396 ;
        time = UNLIMITED ; // (6 currently)
        level = 5 ;
variables:
        double bathymetry(latc, lonc) ;
                bathymetry:units = "m" ;
                bathymetry:long_name = "bathymetry" ;
                bathymetry:valid_range = -5., 4000. ;
                bathymetry:_FillValue = -10. ;
                bathymetry:missing_value = -10. ;
        float h(time, level, latc, lonc) ;
                h:units = "m" ;
                h:long_name = "layer thickness" ;
                h:_FillValue = -9999.f ;
                h:missing_value = -9999.f ;
        double latc(latc) ;
                latc:units = "degrees_north" ;
        double level(level) ;
                level:units = "level" ;
        double lonc(lonc) ;
                lonc:units = "degrees_east" ;
        float temp(time, level, latc, lonc) ;
                temp:units = "degC" ;
                temp:long_name = "temperature" ;
                temp:valid_range = -2.f, 40.f ;
                temp:_FillValue = -9999.f ;
                temp:missing_value = -9999.f ;
        double time(time) ;
                time:long_name = "time" ;
                time:units = "seconds since 1996-01-01 00:00:00" ;
```

### C and Fortran libraries
Used to underpin interfaces to other languages such as python (e.g. python package netCDF4)

Include ncdump/ncgen to convert to and from human readable format.

### nco tools http://nco.sourceforge.net/nco.html
Command line utilities to extract, create and operate data in netCDF files.

```
    > ncks -v u_wind -d lat,50.,51. -d lon,0.,5 inputfile.nc outputfile.nc
```

### cdo tools
Another powerful CLI utility https://code.mpimet.mpg.de/projects/cdo/

### Viewers
ncdump, ncview, panoply, pyncview, etc.

### Readable by many software tools
ArcGIS, QGIS, Surfer, Ferret, Paraview etc.


# References

* xarray [docs](http://xarray.pydata.org/en/stable/) 
* netCDF4 [docs](https://unidata.github.io/netcdf4-python/netCDF4/index.html)
* Stephan Hoyer's [ECMWF talk](https://docs.google.com/presentation/d/16CMY3g_OYr6fQplUZIDqVtG-SKZqsG8Ckwoj2oOqepU/edit#slide=id.g2b68f9254d_1_27)